In [2]:
pip install cohere --upgrade

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install  pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import cohere

# 1. Carga de datos
df_mc = pd.read_csv("C:\\Users\\dairo\\Documents\\universidad\\Especializacion analitica estrategica de datos\\Analitica para la toma de decisiones\\Proyecto\\main\\Projecto_TRM-main\\Datos\\predicciones_bitcoinMontecarlo.csv", parse_dates=["Fecha"])
df_xgb    = pd.read_csv("C:\\Users\\dairo\\Documents\\universidad\\Especializacion analitica estrategica de datos\\Analitica para la toma de decisiones\\Proyecto\\main\\Projecto_TRM-main\\Datos\\predicciones_XGBoost.csv",         parse_dates=["Fecha"])

In [6]:
df_mc.head()

,Fecha,Prediccion,Cambio_7d,Cambio_1d
0,2025-04-07,79235.335938,-4.014078,1.305195
1,2025-04-08,76271.953125,-10.446525,-3.739976
2,2025-04-09,82573.953125,0.106979,8.262539
3,2025-04-10,79725.796875,-4.063678,-3.449219
4,2025-04-11,82945.039062,-1.071952,4.037893


In [7]:
df_xgb = df_xgb.rename(columns={"Predicción": "Prediccion"})
df_xgb.head()

,Fecha,Prediccion,Cambio_7d,Cambio_1d
0,2025-04-06,83955.670,1.968978,0.539939
1,2025-04-07,85497.270,3.571651,1.836209
2,2025-04-08,84268.820,-1.057133,-1.436833
3,2025-04-09,84663.805,2.640571,0.468719
4,2025-04-10,84550.580,1.742119,-0.133737


In [8]:
# 2. Función para obtener las predicciones de una fecha dada
def get_predictions_for_date(fecha_str):
    fecha = pd.to_datetime(fecha_str)
    # Filtrar cada dataset
    mc_row  = df_mc[df_mc["Fecha"] == fecha]
    xgb_row = df_xgb[df_xgb["Fecha"] == fecha]
    if mc_row.empty or xgb_row.empty:
        raise ValueError(f"No hay predicciones para la fecha {fecha_str}")
    return {
        "montecarlo": float(mc_row["Prediccion"].iloc[0]),
        "xgboost":    float(xgb_row["Prediccion"].iloc[0])
    }


In [9]:
# 3. Construcción del prompt para el LLM
def build_prompt(fecha_str, preds: dict):
    return f"""
Eres un analista financiero experto en criptomonedas.
Tienes dos predicciones para el precio de Bitcoin el día {fecha_str}:
  • Predicción Monte Carlo: USD {preds['montecarlo']:.2f}
  • Predicción XGBoost:    USD {preds['xgboost']:.2f}

Contexto adicional:
  • Precio actual de Bitcoin (spot): USD {{precio_spot_actual}}
  • Tu objetivo es maximizar rentabilidad y controlar riesgo.

Según estas predicciones, ¿qué recomendarías hacer ese día?
  - “Vender” si crees que el precio va a bajar,
  - “Comprar” si va a subir,
  - o “Mantener” si no hay una señal clara.
Explica brevemente tu razonamiento (1–2 frases).
"""

In [10]:
# 4. Llamada a la API de Cohere
def recomendar_para_fecha(fecha_str, api_key, precio_spot_actual=None):
    print("Precio actual " + str(precio_spot_actual) )
    preds = get_predictions_for_date(fecha_str)
    prompt = build_prompt(fecha_str, preds)
    # Si tienes el precio spot, lo insertas
    if precio_spot_actual is not None:
        prompt = prompt.replace("{precio_spot_actual}", f"{precio_spot_actual:.2f}")
    else:
        prompt = prompt.replace("• Precio actual de Bitcoin (spot): USD {precio_spot_actual} ", "")

    # Inicializa el cliente Cohere
    co = cohere.ClientV2(api_key)

    # Llamada al modelo
    response = co.chat(
        model="command-a-03-2025",
        messages=[{"role": "user", "content": prompt.strip()}]
    )

    # Resultado
    return response

In [12]:
pip install yfinance 

     -------------------------------------- 109.8/109.8 kB 6.2 MB/s eta 0:00:00
     ---------------------------------------- 3.0/3.0 MB 3.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     -------------------------------------- 186.0/186.0 kB 3.7 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.9-py3-none-any.whl size=139127 sha256=52a30ff53515564516742386dd7e56703b15f387467e482288eaf0c0822d06db
  Stored in directory: c:\users\dairo\appdata\local\pip\cache\wheels\ed\fd\80\48a915968e77972758d473495648213961ac6a251fff460d67
Successfully built peewee
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import yfinance as yf
import datetime

today = datetime.date.today()
# Nueva fecha restar un día
yesterday = today - datetime.timedelta(days=1)
print(today)
btc_data = yf.download("BTC-USD", start=yesterday, end=today, interval="1d")
btc_data.head()
if not btc_data.empty:
  #close_price = btc_data['Close'][0]
  print(btc_data.head())
  #tipo de dato btc_data
  print(type(btc_data))
  # Obtener el precio de cierre segun fecha yesterday
  close_price = btc_data.loc[yesterday.strftime("%Y-%m-%d")]['Close']
  print("Precio de cierre de BTC-USD: ", close_price)
  #print(close_price)
else:
  print("No data found for today.")

[*********************100%***********************]  1 of 1 completed

2025-04-13
Price              Close        High        Low          Open       Volume
Ticker           BTC-USD     BTC-USD    BTC-USD       BTC-USD      BTC-USD
Date                                                                      
2025-04-12  85287.109375  85856.1875  82769.375  83404.515625  24258059104
<class 'pandas.core.frame.DataFrame'>
Precio de cierre de BTC-USD:  Ticker
BTC-USD    85287.109375
Name: 2025-04-12 00:00:00, dtype: float64


In [26]:
print( close_price)

Ticker
BTC-USD    85287.109375
Name: 2025-04-12 00:00:00, dtype: float64


In [31]:
precio_float = close_price["BTC-USD"]
print(precio_float)      # 85287.109375


85287.109375


In [32]:
# Ejemplo de uso:
if __name__ == "__main__":
    API_KEY = "1gyw72nkaPytUjvZw30ntTI5mqy3mwk1MNOzjO2x"
    fecha_consulta = "2025-04-13"
    # precio_spot = 62000.00  # opcional
    respuesta = recomendar_para_fecha(fecha_consulta, API_KEY, precio_spot_actual=precio_float)
    print(respuesta)


Precio actual 85287.109375
id='de34237d-9ee1-451e-8868-2d68512c22fd' finish_reason='COMPLETE' prompt=None message=AssistantMessageResponse(role='assistant', tool_calls=None, tool_plan=None, content=[TextAssistantMessageResponseContentItem(type='text', text='**Recomendación: Mantener**\n\nAmbas predicciones sugieren un precio ligeramente superior al actual, pero la diferencia es mínima y no hay una señal clara de una subida significativa. Dado que el objetivo es maximizar la rentabilidad y controlar el riesgo, mantener la posición actual parece la opción más prudente, evitando costos de transacción innecesarios y posibles pérdidas si el precio no se mueve como se predice.')], citations=None) usage=Usage(billed_units=UsageBilledUnits(input_tokens=185.0, output_tokens=86.0, search_units=None, classifications=None), tokens=UsageTokens(input_tokens=680.0, output_tokens=88.0)) logprobs=None
